In [ ]:
from ipywidgets import widgets
import pandas as pd
from io import BytesIO

# Create the file upload widget
uploader = widgets.FileUpload(
    accept='.xlsx',
    multiple=False
)

# Create an output widget to display results
output = widgets.Output()

# Global variable to store the DataFrame
df = pd.DataFrame()

# Define a function to handle file upload
def on_upload_change(change):
    global df  # Use global to make df accessible outside the function
    with output:
        output.clear_output()
        # Check if a file was uploaded
        if uploader.value:
            # Get the uploaded file (it's a tuple)
            uploaded_file = uploader.value[0]
            # Read the file content
            content = uploaded_file['content']
            # Convert to DataFrame with UTF-8 encoding
            df = pd.read_excel(BytesIO(content), engine='openpyxl')
            print("File uploaded successfully!")
            print(f"DataFrame shape: {df.shape}")
            print("\nFirst few rows:")
            print(df.head())

# Attach the handler to the upload widget
uploader.observe(on_upload_change, names='value')

# Display the widgets
display(uploader)
display(output)

# Now you can use 'df' in subsequent cells after uploading

FileUpload(value=(), accept='.xlsx', description='Upload')

Output()

In [233]:


rename_dict = {
  '訂單編號' : 'sub',
  '訂購客戶' : 'customer',
  '負責業務員' : 'creator',
  '接單單位' : 'dept_sales',
  '開單日期' : 'date_created',
  '理貨單位' : 'dept_fulfillment',
  '出貨單位' : 'dept_shipping',
  '配送方式' : 'transport',
  '溫層' : 'thermo',
  '送貨日期' : 'date_delivery',
  '送貨時間' : 'time_delivery',
  '送貨地址' : 'address',
  '收貨人' : 'recipient',
  '連絡電話(日)' : 'telephone_day',
  '收貨人手機' : 'mobile',
  '送貨備註一' : 'comment_1',
  '送貨備註二' : 'comment_2',
  '送貨備註三' : 'comment_3',
  '是否結案' : 'status',
  '明細項次' : 'item_index',
  '品牌' : 'brand',
  '產品編號' : 'sku',
  '產品名稱' : 'product',
  '訂購量' : 'qty',
  '商品備註' : 'comment',
  '自組產品編號' : 'custom_sku',
  '自組產品名稱' : 'custom_product',
  '明細數量' : 'custom_qty',
}

# class App_Item(TypedDict):
#     sku: str
#     qty: int
#     brand: str
#     unit: str
#     product: str
#     category: str
#     custom_gift_box: bool

# class App_Order(TypedDict):
#     sub: str
#     customer_no: str
#     customer: str
#     recipient: str
#     date_shipping: Optional[datetime.datetime]
#     date_delivery: datetime.datetime
#     time_delivery_start: Any
#     time_delivery_end: Any
#     status: str
#     dept_sales: str
#     dept_fulfillment: str
#     dept_shipping: str
#     dept_pickup: str
#     postnumber: str
#     address: str
#     transport: str
#     thermo: str
#     comment_order: Any
#     comment_1: Any
#     comment_2: Any
#     comment_3: Any
#     telephone_day: Any
#     telephone_evening: Any
#     mobile: Any
#     data: List[App_Item] = []
#     tracking: List = []

In [234]:
from pydantic import TypeAdapter
from typing import TypedDict, Optional, Any, List
from datetime import datetime

class Address(TypedDict):
    Search: str
    PostNumber: str


In [270]:
from datetime import datetime 

upload_df = df.fillna('')
upload_df = upload_df.rename(columns=rename_dict)

date_format = '%Y/%m/%d'
time_format = '%H%M'

upload_df['main'] = upload_df.loc[:, 'sub'].apply(lambda x: x[:9])

upload_df['customer_no'] = upload_df.loc[:, 'customer'].apply(lambda x: x[:7])
upload_df.loc[:, 'customer'] = upload_df.loc[:, 'customer'].apply(lambda x: x[7:])

upload_df['dept_pickup'] = upload_df.loc[:, 'dept_shipping'].replace('0001 總倉(2F廠務辦公室)', '')
upload_df['dept_pickup'] = upload_df.loc[:, 'dept_shipping'].replace('0001 總倉(2F廠務辦公室)', '')
upload_df['date_delivery'] = upload_df.loc[:, 'date_delivery'].apply(lambda x: datetime.strptime(x, date_format))
upload_df['date_created'] = upload_df.loc[:, 'date_created'].apply(lambda x: datetime.strptime(x, date_format))

upload_df['time_delivery_start'] = upload_df.loc[:, 'time_delivery'].copy().apply(lambda x: x[:4])
upload_df['time_delivery_end'] = upload_df.loc[:, 'time_delivery'].copy().apply(lambda x: x[-4:])
upload_df['postnumber'] = ''
upload_df['custom_gift_box'] = upload_df['custom_qty'].apply(lambda x: True if not x else False)

print(upload_df.tail())

                sub        customer creator dept_sales date_created  \
12658  O26001429001   Tara @ Taipei  網購公共帳號    9000 網購   2026-01-11   
12659  O26001429001   Tara @ Taipei  網購公共帳號    9000 網購   2026-01-11   
12660  O26001429001   Tara @ Taipei  網購公共帳號    9000 網購   2026-01-11   
12661  O26001997001             陳又菱  網購公共帳號    9000 網購   2026-01-14   
12662  O26002019001        珊(第三方登入)  網購公共帳號    9000 網購   2026-01-15   

       dept_fulfillment dept_shipping transport thermo date_delivery  ...  \
12658  0001 總倉(2F廠務辦公室)   0002 師大泰順門市                     2026-02-01  ...   
12659  0001 總倉(2F廠務辦公室)   0002 師大泰順門市                     2026-02-01  ...   
12660  0001 總倉(2F廠務辦公室)   0002 師大泰順門市                     2026-02-01  ...   
12661  0001 總倉(2F廠務辦公室)     0003 板橋門市                     2026-02-01  ...   
12662  0001 總倉(2F廠務辦公室)  0005 5號起司高鐵店                     2026-02-01  ...   

      custom_sku custom_product custom_qty       main customer_no  \
12658                                    

In [271]:
orders = upload_df[[
    'sub',
    'main',
    'date_created',
    'customer_no',
    'creator',
    'customer',
    'dept_sales',
    'dept_fulfillment',
    'dept_shipping',
    'dept_pickup',
    'transport',
    'thermo',
    'status',
    'time_delivery_start',
    'time_delivery_end',
    'date_delivery',
    'comment_1',
    'comment_2',
    'comment_3',
    'telephone_day',
    'mobile',
    'recipient',
]]
order_address = upload_df.query('transport == "黑貓宅急便"')[[
    'sub',
    'address',
    'postnumber'
]]
order_items = upload_df[[
    'sub',
    'sku',
    'product',
    'qty',
    'comment'
]]
order_custom = upload_df[[
    'sub',
    'custom_sku',
    'custom_product',
    'custom_qty',
]]
order_tracking = upload_df[[
    'sub'
]]


In [ ]:

import asyncio
from shared.tcat_api import *
order_address = order_address.drop_duplicates().reset_index(drop=True)

batch_size = 100
batch_list = []


for i in range(0, len(order_address), batch_size):
    batch_addresses = order_address.iloc[i : i + batch_size]
    batch_list.append(batch_addresses)

async with httpx.AsyncClient() as client:
    tasks = [Batch_Label_PostNumber(client, batch) for batch in batch_list]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    

[{'Search': '241新北市三重區自強路一段196號1樓', 'PostNumber': '40-279-12-D'}, {'Search': '221新北市汐止區復興路26號', 'PostNumber': '29-067-75-B'}, {'Search': '300新竹市北區湳雅街91-2號', 'PostNumber': '31-291-67-G'}, {'Search': '337桃園市大園區中正西路27號', 'PostNumber': '30-381-15-C'}, {'Search': '310新竹縣竹東鎮長春路一段266號', 'PostNumber': '31-314-61-K'}, {'Search': '300新竹縣寶山鄉園區三路123號', 'PostNumber': '31-299-26-C'}, {'Search': '338桃園市蘆竹區南竹路１段59號', 'PostNumber': '30-385-14-F'}, {'Search': '333桃園市龜山區文化二路34巷14弄27號', 'PostNumber': '21-333-21-C'}, {'Search': '500彰化縣彰化市成功路322號', 'PostNumber': '64-502-21-E'}, {'Search': '600嘉義市西區中山路441號', 'PostNumber': '61-582-61-F'}, {'Search': '803高雄市鹽埕區五福四路162號', 'PostNumber': '81-812-15-F'}, {'Search': '100台北市中正區八德路一段104號', 'PostNumber': '11-116-01-B'}, {'Search': '320桃園市中壢區自強一路5-1號', 'PostNumber': '41-342-13-G'}, {'Search': '220新北市板橋區忠孝路52號', 'PostNumber': '40-666-60-J'}, {'Search': '220新北市板橋區介壽街2號', 'PostNumber': '40-663-32-A'}, {'Search': '104台北市中山區中山北路二段160號', 'PostNumber': '41-093-84-A'}, {'Searc

In [ ]:
flat_list = sum(results, [])
postnumber_df = pd.DataFrame(flat_list)
postnumber_df = postnumber_df.rename(columns = {'Search' : 'address', 'PostNumber' : 'postnumber'})


2776
                      address   postnumber
0     104台北市中山區民生東路二段83號3樓之45  41-091-11-E
1              台北市大安區和平東路三段6號  11-172-23-H
2             台北市中正區新生南路一段90號  11-115-05-B
3           台北市信義區松高路12號A8 B2  19-153-66-A
4              台北市大安區敦化南路二段5號  11-171-06-A
...                       ...          ...
2771         744台南市新市區復興路669號  75-703-13-A
2772     300新竹市東區中華路三段9號11樓之5  31-292-71-A
2773          420台中市豐原區陽明街36號  69-424-05-B
2774       555南投縣魚池鄉魚池街557巷4號  67-555-55-E
2775     406台中市北屯區景賢路179號7樓之5  56-464-95-C

[2776 rows x 2 columns]


In [ ]:

final_order_address = pd.merge(order_address.drop(columns=['postnumber']), postnumber_df.drop_duplicates(), how='left', on='address')
print(final_order_address)


                      address   postnumber
0     104台北市中山區民生東路二段83號3樓之45  41-091-11-E
1              台北市大安區和平東路三段6號  11-172-23-H
2             台北市中正區新生南路一段90號  11-115-05-B
3           台北市信義區松高路12號A8 B2  19-153-66-A
4              台北市大安區敦化南路二段5號  11-171-06-A
...                       ...          ...
2771         744台南市新市區復興路669號  75-703-13-A
2772     300新竹市東區中華路三段9號11樓之5  31-292-71-A
2773          420台中市豐原區陽明街36號  69-424-05-B
2774       555南投縣魚池鄉魚池街557巷4號  67-555-55-E
2775     406台中市北屯區景賢路179號7樓之5  56-464-95-C

[2776 rows x 2 columns]
               sub                  address   postnumber
0     O25010578001  104台北市中山區民生東路二段83號3樓之45  41-091-11-E
1     O25011458001           台北市大安區和平東路三段6號  11-172-23-H
2     O25011458002          台北市中正區新生南路一段90號  11-115-05-B
3     O25011458003        台北市信義區松高路12號A8 B2  19-153-66-A
4     O25011458004           台北市大安區敦化南路二段5號  11-171-06-A
...            ...                      ...          ...
2771  O26002097001         744台南市新市區復興路669號  75-703-13-A
277

In [253]:
import asyncpg
import pandas as pd
from typing import List

async def update_table_from_dataframe(
    df: pd.DataFrame,
    table_name: str,
    key_columns: List[str],
    host: str = 'localhost',
    port: int = 5432,
    database: str = 'your_database',
    user: str = 'your_user',
    password: str = 'your_password'
):
    """
    Update a PostgreSQL table using a DataFrame with asyncpg.
    
    Parameters:
    -----------
    df : pd.DataFrame
        The DataFrame containing the data to update
    table_name : str
        Name of the table to update
    key_columns : List[str]
        List of column names to use as the key for matching rows (e.g., ['id'])
    host, port, database, user, password : str
        PostgreSQL connection parameters
    
    Returns:
    --------
    int : Number of rows updated
    """
    
    # Connect to PostgreSQL
    conn = await asyncpg.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    
    try:
        # Get all columns except the key columns
        update_columns = [col for col in df.columns if col not in key_columns]
        
        # Build the UPDATE query
        set_clause = ", ".join([f"{col} = ${i+1}" for i, col in enumerate(update_columns)])
        where_clause = " AND ".join([f"{key} = ${len(update_columns) + i + 1}" 
                                      for i, key in enumerate(key_columns)])
        
        query = f"""
            UPDATE {table_name}
            SET {set_clause}
            WHERE {where_clause}
        """
        
        print(f"Query: {query}")
        
        # Prepare the data for batch update
        update_count = 0
        
        # Execute updates in a transaction for better performance
        async with conn.transaction():
            for _, row in df.iterrows():
                # Prepare values: update columns first, then key columns
                values = [row[col] for col in update_columns] + [row[key] for key in key_columns]
                
                result = await conn.execute(query, *values)
                # Extract number of rows affected from result (e.g., "UPDATE 1")
                if result.split()[-1].isdigit():
                    update_count += int(result.split()[-1])
        
        print(f"Successfully updated {update_count} rows")
        return update_count
        
    finally:
        # Close the connection
        await conn.close()

from database.info import DB_URL
# Alternative: Batch update with executemany (more efficient for large datasets)
async def batch_update_table_from_dataframe(
    df: pd.DataFrame,
    table_name: str,
    key_columns: List[str],
    # host: str = 'localhost',
    # port: int = 5432,
    # database: str = 'your_database',
    # user: str = 'your_user',
    # password: str = 'your_password'
):
    """
    Batch update a PostgreSQL table using a DataFrame with asyncpg (more efficient).
    """
    

    pool = await asyncpg.create_pool(DB_URL)

    async with pool.acquire() as conn:
    
        try:
            # Get all columns except the key columns
            update_columns = [col for col in df.columns if col not in key_columns]
            
            # Build the UPDATE query
            set_clause = ", ".join([f"{col} = ${i+1}" for i, col in enumerate(update_columns)])
            where_clause = " AND ".join([f"{key} = ${len(update_columns) + i + 1}" 
                                        for i, key in enumerate(key_columns)])
            
            query = f"""
                UPDATE {table_name}
                SET {set_clause}
                WHERE {where_clause}
            """
            
            print(f"Query: {query}")
            
            # Prepare data as list of tuples
            data = []
            for _, row in df.iterrows():
                values = tuple([row[col] for col in update_columns] + [row[key] for key in key_columns])
                data.append(values)
            
            # Execute batch update
            async with conn.transaction():
                await conn.executemany(query, data)
            
            print(f"Successfully updated {len(data)} rows")
            return len(data)
            
        finally:
            await conn.close()

In [ ]:
await batch_update_table_from_dataframe(df=final_order_address, table_name='order_address', key_columns=['sub'])
# await batch_update_table_from_dataframe(df=orders, table_name='orders', key_columns=['sub'])

Query: 
                UPDATE orders
                SET main = $1, date_created = $2, customer_no = $3, creator = $4, customer = $5, dept_sales = $6, dept_fulfillment = $7, dept_shipping = $8, dept_pickup = $9, transport = $10, thermo = $11, status = $12, time_delivery_start = $13, time_delivery_end = $14, date_delivery = $15, comment_1 = $16, comment_2 = $17, comment_3 = $18, telephone_day = $19, mobile = $20, recipient = $21
                WHERE sub = $22
            


DataError: invalid input for query argument $2 in element #0 of executemany() sequence: Timestamp('2025-11-06 00:00:00') (expected str, got Timestamp)